In [1]:
import os
import zarr
import random
import json
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch
import torchvision.transforms.functional as F
import random
import sys
from collections import defaultdict

warnings.filterwarnings("ignore")
sys.path.append("./src/")

from src.config import CFG
from src.dataloader import (
    read_zarr,
    read_info_json,
    scale_coordinates,
    create_dataset,
    create_segmentation_map,
    EziiDataset,
    drop_padding,
)
from src.network import UNet_2D, aug
from src.utils import save_images
from src.metric import score, create_cls_pos, create_cls_pos_sikii, create_df

sample_submission = pd.read_csv("../../inputs/sample_submission.csv")

In [2]:
valid_dataset = EziiDataset(
    exp_names=CFG.valid_exp_names,
    # exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.valid_zarr_types,
    train=True,
)

from tqdm import tqdm

# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False)

for row in tqdm(valid_loader):
    normalized_tomogram = row["normalized_tomogram"]
    break

[('TS_86_3', 'denoised'), ('TS_6_6', 'denoised')]


  0%|          | 0/2 [00:00<?, ?it/s]


In [3]:
class PadToSize(nn.Module):
    def __init__(self, resolution):
        super().__init__()
        if resolution == "0":
            self.size = 640
        elif resolution == "1":
            self.size = 320
        elif resolution == "2":
            self.size = 160

    def forward(self, x):
        return F.pad(x, (0, 0, self.size - x.shape[-1], self.size - x.shape[-2]))

In [4]:
model = UNet_2D().to("cuda")
model.eval()
model.load_state_dict(torch.load("./best_model.pth"))


optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(
    weight=torch.tensor([0.5, 32, 32, 32, 32, 32, 32]).to("cuda")
)
# criterion = DiceLoss()

best_model = None
best_loss = np.inf
batch_size = 4

valid_loss = []
valid_pred_tomogram = defaultdict(list)
valid_gt_tomogram = defaultdict(list)
model.eval()
tq = tqdm(range(len(valid_loader) * normalized_tomogram.shape[0]))
for data in valid_loader:
    exp_name = data["exp_name"][0]
    tomogram = data["normalized_tomogram"].to("cuda")
    segmentation_map = data["segmentation_map"].to("cuda").long()

    for i in range(tomogram.shape[1]):
        input_ = tomogram[:, i].unsqueeze(0)
        gt = segmentation_map[:, i]

        input_ = PadToSize(CFG.resolution)(input_)
        gt = PadToSize(CFG.resolution)(gt)
        output = model(input_)
        loss = criterion(output, gt)

        valid_loss.append(loss.item())
        tq.set_description(f"Loss: {np.mean(valid_loss)}")
        tq.update(1)

        output = drop_padding(output, CFG.resolution)

        valid_pred_tomogram[exp_name].append(output.cpu().detach().numpy())
        valid_gt_tomogram[exp_name].append(gt.cpu().detach().numpy())
tq.close()

Loss: 0.8741244813224868: : 184it [00:09, 20.34it/s]                     


In [5]:
valid_pred_tomogram.keys()

dict_keys(['TS_86_3', 'TS_6_6'])

In [6]:
def create_gt_df(base_dir, exp_names):
    result_df = None
    particle_names = CFG.particles_name

    for exp_name in exp_names:
        for particle in particle_names:
            np_corrds = read_info_json(
                base_dir=base_dir, exp_name=exp_name, particle_name=particle
            )  # (n, 3)
            # 各行にexp_nameとparticle_name追加
            particle_df = pd.DataFrame(np_corrds, columns=["z", "y", "x"])
            particle_df["experiment"] = exp_name
            particle_df["particle_type"] = particle

            if result_df is None:
                result_df = particle_df
            else:
                result_df = pd.concat([result_df, particle_df], axis=0).reset_index(
                    drop=True
                )

    result_df = result_df.reset_index()  # index	experiment	particle_type	x	y	z
    result_df = result_df[["index", "experiment", "particle_type", "x", "y", "z"]]

    return result_df


gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.valid_exp_names)
gt_df = gt_df[gt_df["particle_type"] != "beta-amylase"].reset_index(drop=True)
gt_df

,index,experiment,particle_type,x,y,z
0,0,TS_86_3,apo-ferritin,3870.343,4952.714,1261.600
1,1,TS_86_3,apo-ferritin,4130.897,5422.292,501.860
2,2,TS_86_3,apo-ferritin,2735.000,4668.447,520.291
3,3,TS_86_3,apo-ferritin,2649.615,4690.615,600.923
4,4,TS_86_3,apo-ferritin,2665.353,4810.641,612.019
...,...,...,...,...,...,...
340,363,TS_6_6,virus-like-particle,2609.876,4569.876,1169.759
341,364,TS_6_6,virus-like-particle,2213.287,4135.017,1286.851
342,365,TS_6_6,virus-like-particle,3303.905,5697.825,789.744
343,366,TS_6_6,virus-like-particle,1008.748,5949.213,1077.303


In [41]:
# pred_df = pd.read_csv("../../inputs/train_submission.csv")
all_pred_df = None

for exp_name in CFG.valid_exp_names:
    pred_tomogram = valid_pred_tomogram[exp_name]
    pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
    pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

    pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
        pred_tomogram, sikii_dict=CFG.initial_sikii
    )
    pred_df = create_df(pred_Ascale_pos, exp_name)
    # pred_df = create_df(pred_cls_pos, exp_name)

    if all_pred_df is None:
        all_pred_df = pred_df
    else:
        all_pred_df = pd.concat([all_pred_df, pred_df], axis=0).reset_index(drop=True)

pred_df = all_pred_df[all_pred_df["particle_type"] != "beta-amylase"].reset_index(
    drop=True
)

pred_df

,experiment,particle_type,x,y,z
0,TS_86_3,apo-ferritin,3703.333333,873.333333,493.333333
1,TS_86_3,apo-ferritin,3139.999630,3140.001490,910.000274
2,TS_86_3,beta-galactosidase,3139.976326,3139.957965,910.015876
3,TS_86_3,beta-galactosidase,2986.000000,4414.000000,220.000000
4,TS_86_3,beta-galactosidase,3138.000000,5578.000000,226.000000
...,...,...,...,...,...
445,TS_6_6,virus-like-particle,1523.200000,1383.600000,1262.000000
446,TS_6_6,virus-like-particle,795.733333,3320.800000,1331.733333
447,TS_6_6,virus-like-particle,3380.000000,960.000000,1440.000000
448,TS_6_6,virus-like-particle,3490.000000,1060.000000,1440.000000


In [42]:
pred_df["experiment"].unique(), gt_df["experiment"].unique()

(array(['TS_86_3', 'TS_6_6'], dtype=object),
 array(['TS_86_3', 'TS_6_6'], dtype=object))

In [43]:
score(
    pred_df,
    gt_df,
    row_id_column_name="index",
    distance_multiplier=1,
    beta=4,
)

0.010146782628316588